In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import tensorflow as tf

from data import example_to_tensor, normalize, add_channel_axis
from plot import plot_slice, plot_animated_volume
from config import data_root_dir

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [2]:
def min_max_normalize(scan):
    min_value = tf.reduce_min(scan)
    max_value = tf.reduce_max(scan)
    return normalize(scan, min_value, max_value)

In [3]:
tfrecord_fnames = [str(p) for p in Path(data_root_dir).glob("LUNA16/*.tfrecord")]
dataset = (
    tf.data.TFRecordDataset(tfrecord_fnames).map(
        example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    .map(min_max_normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
dataset

<ParallelMapDataset shapes: (None, None, None, 1), types: tf.float32>

In [ ]:
sum(1 for _ in dataset)

In [5]:
i = next(iter(dataset.skip(100)))
i.shape

TensorShape([48, 256, 256, 1])

In [6]:
#plt.hist(i.numpy().flatten(), bins=100);

In [7]:
plot_animated_volume(i)